In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import altair as alt
import pandas as pd
from vega_datasets import data
import datetime as dt
from altair import datum

In [ ]:
alt.renderers.enable('default')
pn.extension('vega')

In [ ]:
covid_data_scandinavia = pd.read_csv("data/covid_data_scandinavia.csv")
covid_data_scandinavia['Date'] = pd.to_datetime(covid_data_scandinavia.Date)
covid_data_scandinavia

In [ ]:
long_form_scandinavia = pd.read_csv('long_form_scandinavia.csv')
long_form_scandinavia['Date'] = pd.to_datetime(long_form_scandinavia.Date)
long_form_scandinavia

In [ ]:
alt.Chart(long_form_scandinavia).mark_line().encode(
    x='Date',
    y='Cases',
    color='Country')

In [ ]:
# Create tickers for selection options
tickers = ['Denmark', 'Finland', 'Norway', 'Sweden']
ticker = pn.widgets.Select(name='Country', options=tickers)

# Create range slider for date ranges
date_range_slider = pn.widgets.DateRangeSlider(name='Date Range', 
                                               start=long_form_scandinavia['Date'].min(),
                                               end=long_form_scandinavia['Date'].max(),
                                               value=(long_form_scandinavia['Date'].min(), long_form_scandinavia['Date'].max())
                                              )

title = '### Confirmed Cases'
subtitle = 'This dashboard allows you to select a country and date range to see total confirmed cases.'

@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range):
    # Load data
    df = long_form_scandinavia 
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Create date filter using values from range slider
    # Store first and last date range slider
    start_date = date_range_slider.value[0]
    end_date = date_range_slider.value[1]
    
    # Create filter mask for the dataframe
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
    df = df.loc[mask]
    
    # Create chart object
    chart = alt.Chart(df).mark_area(color="#0c1944", opacity=0.8).encode(x='Date', y='Cases', tooltip=alt.Tooltip(['Date','Cases'])).transform_filter(
        (datum.symbol == ticker)
    )
    
    return chart

dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot
)

dashboard.servable()

In [ ]:
# set a title for your dashboard
title = '## Stock Price Dashboard'

subtitle = 'This dashboard allows you to select a company and date range to see stock prices.'

# create list of company names (tickers)
tickers = ['AAPL', 'GOOG', 'IBM', 'MSFT']

# this creates the dropdown widget
ticker = pn.widgets.Select(name='Company', options=tickers)

# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
    value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)

# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below Altair chart as filters
@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range): # start function

    # Load and format the data
    df = source # define df
    df['date'] = pd.to_datetime(df['date']) # format date as datetime
    
    # create a date filter that uses values from the date range slider
    start_date = date_range_slider.value[0] # store the first date range slider value in a var
    end_date = date_range_slider.value[1] # store the end date in a var
    mask = (df['date'] > start_date) & (df['date'] <= end_date) # create filter mask for the dataframe
    df = df.loc[mask] # filter the dataframe
    
    # create the Altair chart object
    chart = alt.Chart(df).mark_area(color="#0c1944", opacity=0.8).encode(x='date', y='price', tooltip=alt.Tooltip(['date','price'])).transform_filter(
        (datum.symbol == ticker) # this ties in the filter from the dropdown selection
    )
    
    return chart

# create the Panel object, passing in all smaller objects

dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot # draw chart function!
)

dashboard.servable()